In [1]:
import xarray as xr
import data_utils
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.animation as animation
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
#uncomment commented lines when test data is ready

site = 'US-A32'

path_to_starfm_istsg = f'gs://sundai-satellite-data/benchmark/starfm_istsg_ndvi_{site}.nc'
#path_to_deep_learning = f'gs://sundai-satellite-data/benchmark/deeplearning_{site}.nc'

benchmark = data_utils.load_dataset(path_to_starfm_istsg)
#test = data_utils.load_dataset(path_to_deep_learning)

benchmark = benchmark.sel(time=slice(pd.to_datetime('2017-01-01'), pd.to_datetime('2024-12-31')))


In [3]:
#test['ndvi'] = (test['nir08'] - test['red'])/ (test['nir08'] + test['red'])

benchmark_mean = benchmark.mean(dim=['y', 'x'])
#test_mean = test.mean(dim=['y', 'x'])

In [4]:
pal = sns.color_palette('pastel',5)

In [5]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.animation as animation
import seaborn as sns
import pandas as pd
import numpy as np

# Assuming you already have these xarrays:
# starfm, mod_filled, f1 -> with dims ('time', 'y', 'x')
# starfm_mean, mod_filled_mean, f1_mean -> with dims ('time',)

# Compute vmin and vmax
vmin = benchmark.ndvi.quantile(0.2).values.item()
vmax = benchmark.ndvi.quantile(0.98).values.item()

# Get sorted time steps
time_steps = pd.to_datetime(benchmark.time.values)

# Set up the figure
fig = plt.figure(figsize=(10, 6))
gs = gridspec.GridSpec(2, 2, height_ratios=[1, 1])
ax_top = fig.add_subplot(gs[0, :])
ax_bottom1 = fig.add_subplot(gs[1, 0])
ax_bottom2 = fig.add_subplot(gs[1, 1])

# Static line plots (do once)
sns.lineplot(x=benchmark_mean.time, y=benchmark_mean['ndvi'].values, label='benchmark', alpha=0.6, ax=ax_top)
#sns.scatterplot(x=benchmark_mean.time, y=benchmark_mean['ndvi'].values, label='benchmark', alpha=0.6, ax=ax_top)

#sns.lineplot(x=test_mean.time, y=test_mean['ndvi'].values, label='test', alpha=0.6, ax=ax_top)

ax_top.legend(ncol=2, frameon=False, loc=(0, 1))
ax_top.set_xlabel('')
ax_top.set_ylabel('NDVI')

# Placeholders for dynamic content
scatter1 = ax_top.scatter([], [], color='blue')
scatter2 = ax_top.scatter([], [], color='orange')

im1 = ax_bottom1.imshow(np.zeros_like(benchmark.isel(time=0).ndvi.values), vmin=vmin, vmax=vmax)

im2 = ax_bottom2.imshow(np.zeros_like(benchmark.isel(time=0).ndvi.values), vmin=vmin, vmax=vmax) #replace with test

ax_bottom1.set_title("benchmark", fontsize=10)
ax_bottom2.set_title("test", fontsize=10)

# Turn off ticks for image subplots
for ax in [ax_bottom1, ax_bottom2]:
    ax.axis('off')

# Animation update function
def update(frame_idx):
    t = time_steps[frame_idx]

    # Update scatter points
    scatter1.set_offsets([[t, float(benchmark_mean.sel(time=t).ndvi.values)]])
    scatter2.set_offsets([[t, float(benchmark_mean.sel(time=t).ndvi.values)]]) #replace with test

    # Update images
    im1.set_array(benchmark.sel(time=t).ndvi.values)
    im2.set_array(benchmark.sel(time=t).ndvi.values) #replace with test
    

    return [scatter1, scatter2, im1, im2]

# Create animation
ani = animation.FuncAnimation(fig, update, frames=len(time_steps), blit=False)

# Save to MP4
ani.save(f"sundai_benchmark_{site}.mp4", writer='ffmpeg', fps=8, dpi=150)

plt.close()
